# 1.将text划分为token

In [2]:
'''Load data'''
import os
import urllib.request

if not os.path.exists("./data/the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "./data/the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

with open("./data/the-verdict.txt", 'r', encoding = 'utf-8') as f:
    raw_text = f.read()

print(len(raw_text))
print(raw_text[0:99])

20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [3]:
'''Process the text using regular expression'''
import re
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

print(preprocessed[0:30])
print(len(preprocessed)) #这就是token的数量

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']
4690


# 2.将Token映射为Token IDs 

In [4]:
'''Build vocabulary containing all these tokens'''
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size) 

# Note
# 构建字典的时候，需要先去重，然后按照字母表的顺序排序

1130


In [5]:
vocab = {token:integer for integer,token in enumerate(all_words)}

for i, item in enumerate(vocab.items()):
    print(item)
    if i == 20:
        break

# Note:
# token作为字典的key的原因是，现在是token -> token IDs，是拿token去查

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)


In [6]:
'''Put all these operations into a tokenizer class'''
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab #词典是提前建立好的，tokenizer只是拿过来用了
        self.int_to_str = {i:s for s, i in vocab.items()}
    
    def encode(self, text):
        '''将token转换为token ID'''
        # 查词典之前需要先对文本进行处理
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]

        #在vocab中取出token对应的token ID
        ids = [self.str_to_int[s] for s in preprocessed]

        return ids

    def decode(self, ids):
        '''将token ID转换为文本'''
        text = " ".join([self.int_to_str[i] for i in ids])

        # 将特殊字符前的空格替换
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)

        return text

In [7]:
'''Tokenize text using the tokenizer'''

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""

tokenizer = SimpleTokenizerV1(vocab)
ids = tokenizer.encode(text)

print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [8]:
'''Decode the integers back into text'''
print(tokenizer.decode(ids))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


# 3.添加特殊的context tokens


In [ ]:
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text) #Hello在字典中是没有的

KeyError: 'Hello'

In [27]:
'''Adding context tokens <|endoftext|> and <|unk|> into the vocabulary'''
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer, token in enumerate(all_tokens)}

In [28]:
print(len(vocab))

for i,item in enumerate(list(vocab.items())[-5:]):
    print(item)

1132
('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [29]:
'''Replace unknown words in the input text with <|unk|> token'''
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab #词典是提前建立好的，tokenizer只是拿过来用了
        self.int_to_str = {i:s for s, i in vocab.items()}
    
    def encode(self, text):
        '''将token转换为token ID'''
        # 查词典之前需要先对文本进行处理
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        
        # 将输入中，在字典中没有的词，替换为<|unk|>
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]

        #在vocab中取出token对应的token ID
        ids = [self.str_to_int[s] for s in preprocessed]

        return ids

    def decode(self, ids):
        '''将token ID转换为文本'''
        text = " ".join([self.int_to_str[i] for i in ids])

        # 将特殊字符前的空格替换
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)

        return text

In [ ]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

# 使用<|endoftext|>连接不相关的文本，注意在连接的时候，前后一定要加空格！！！
text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [35]:
# 对包含未知单词的文本进行tokenization
print(tokenizer.encode(text))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


In [ ]:
# 对tokenized的结果再de-tokenization
print(tokenizer.decode(tokenizer.encode(text)))

# 可以发现，在de-tokenization的时候，如果是存在字典中未知的token，就会无法解析回到原始的文本了

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


# 4.字节对编码BPE

In [25]:
import importlib
import tiktoken

print(importlib.metadata.version("tiktoken"))

0.9.0


In [ ]:
'''Load the tokenizer'''
tokenizer = tiktoken.get_encoding("gpt2")

# Note：
# gpt2的tokenizer词典一共50257个词

In [ ]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

'''Tokenization'''
integers = tokenizer.encode(text, allowed_special = {"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [28]:
'''De-tokenization'''
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.
